# SVI パート3： ELBO 勾配の推定法

## はじめに


* SVI で興味のあるパラメータは2種類
    * 確率モデルのパラメータ $\theta$
    * 変分分布（ガイド）の変分パラメータ $\phi$
* 究極の目的である、対数エビデンス $\log p(x;\theta)$ 最大化の為に、$\theta$ と $\phi$ に関して ELBO を最大化する
    * ELBO の最大化は $\{ \theta, \phi \}$ で張られるパラメータ空間での勾配降下によって実行する
    * なので、ELBO の勾配 $\nabla_{\theta,\phi} \mathrm{ELBO}$ が欲しい
* 一般の確率モデルと変分分布に対してELBOの勾配を求めるにはどうすればいい？
    * ここでは簡単のため $\theta$ と $\phi$ の区別をなくし、$\phi$ に統一して議論を進める
    * つまり次を求めることを考える
$$
\nabla _ { \phi } \mathbb { E } _ { q _ { \phi } ( \mathbf { z } ) } \left[ f _ { \phi } ( \mathbf { z } ) \right]
$$

### 簡単なケース：再パラメータ化可能な確率変数

$$
\mathbb { E } _ { q _ { \phi } ( \mathbf { z } ) } \left[ f _ { \phi } ( \mathbf { z } ) \right] = \mathbb { E } _ { q ( \epsilon ) } \left[ f _ { \phi } \left( g _ { \phi } ( \epsilon ) \right) \right]
$$

* VAE とかで使われてるテクニック
* 微分可能な関数 $g(\epsilon, \phi)$ で $z$ を求めることができれば簡単に微分できる
    * ただし $\epsilon$ は $\phi$ に関係ない分布 $q(\epsilon)$ に従う
* $\nabla_\phi$ を取ると、期待値計算の中にはいるのでモンテカルロ方による推定ができる

### やっかいなケース： 最パラメータ化できない確率変数

（一般の）ELBOに対する勾配の定義 
$$
\nabla _ { \phi } \mathbb { E } _ { q _ { \phi } ( \mathbf { z } ) } \left[ f _ { \phi } ( \mathbf { z } ) \right] = \nabla _ { \phi } \int q _ { \phi } ( \mathbf { z } ) f _ { \phi } ( \mathbf { z } ) d \mathbf { z } 
$$
に、勾配のチェインルールを使うと
$$
\int \left\{ \left( \nabla _ { \phi } q _ { \phi } ( \mathbf { z } ) \right) f _ { \phi } ( \mathbf { z } ) + q _ { \phi } ( \mathbf { z } ) \left( \nabla _ { \phi } f _ { \phi } ( \mathbf { z } ) \right) \right\} d \mathbf { z } 
$$
となる。

ここで、積分の中で $q(z)$ をまとめることを考える（なぜなら期待値として書けるから）。勾配に対して一般に成り立つ式
$$
\nabla _ { \phi } q _ { \phi } ( \mathbf { z } ) = q _ { \phi } ( \mathbf { z } ) \nabla _ { \phi } \log q _ { \phi } ( \mathbf { z } )
$$
を使用すれば、結局
$$
\mathbb { E } _ { q _ { \phi } ( \mathbf { z } ) } \left[ \left( \nabla _ { \phi } \log q _ { \phi } ( \mathbf { z } ) \right) f _ { \phi } ( \mathbf { z } ) + \nabla _ { \phi } f _ { \phi } ( \mathbf { z } ) \right]
$$
を得る。

* これは REINFORCE 推定量、スコア関数推定量、尤度比推定量として知られる推定量
* $z \sim q(z;\phi)$ をサンプリングするモンテカルロ法によって近似計算可能
* 実装ではこの中身を目的関数としてやる
    * 結局 ELBO の微分をすれば、結局中身の微分（の期待値）になるから
    * こういう目的関数を surrogate objective function という

## ELBO 勾配推定量の分散が大きい問題

* 一般的なモデルと変分分布に対する ELBO 勾配の近似計算方法を確認したが、大抵の場合この近似は高い分散を持つ
    * REINFORCE「ELBO勾配の近似できた！」⇒ 全然違う…
    * （ちなみにREINFORCE は不偏推定量なのでこれを100回とか繰り返すと安定する）
* なので、分散が小さい推定量を得る戦略を考えなければならない
    * 戦略1：コスト関数 $f(\cdot)$ の構造を利用する方法
    * 戦略2：前回の推定値を利用する方法
    
### 戦略1：依存構造を使った分散削減

* 上の推定量の出し方は一般の $f$ に対する議論だったけど、私達が興味があるのは特殊な $f$、つまり ELBO
    * ELBO の依存構造に注目するのがこの戦略
* Rao-Blackwellの定理とよばれる定理を使えば、追加情報（i.e. 十分統計量）を使っtえ不偏推定量の分散を減らせるらしい
    * モンテカルロ計算などで用いられている
    * 詳細は他の資料を参照のこと
* Rao-Blackwell を使うには、`SVI` のインスタンス化の際に `TraceGraph_ELBO` を渡すだけ
    * 計算量が増加するので、むやみに使うもんでもない
    * パラメータ化ができない潜在変数が存在するモデルに対してだけ使えばよくで、大体のモデルでは `Trace_ELBO` で十分
* 使う時の注意としては、`plate` を使用しないと意味がないということ
    * モデルの構造（条件付き独立性）を記述するには `plate` を使うしかない

### 戦略2：ベースライン法

* 2つめの分散低減戦略は baseline と呼ばれる手法
* 期待値が 0 の（ELBO 勾配推定量の普遍性を変えないような）項を追加することを考える
    * 戦略1では、期待値が 0 で分散増加に寄与する項を取り除く事を考えたけど、その反対

目的関数
$$
\log q _ { \phi } \left( \mathbf { z } _ { i } \right) \overline { f _ { \phi } ( \mathbf { z } ) }
$$
を
$$
\log q _ { \phi } \left( \mathbf { z } _ { i } \right) \left( \overline { f _ { \phi } ( \mathbf { z } ) } - b \right)
$$
に変更する。

* これの勾配の期待値は、元の目的関数の勾配期待値と同じだけど、分散が違う
* 美味い具合に $b$ を選ぶことができれば、嬉しい
    * $b$ をベースライン (baseline) という

#### Pyro でのベースライン

* Pyro ではいろんなベースラインを使う方法が提供されている
* ベースラインは任意のパラメータ化不可能な確率変数に付与することが出来るので、確率変数の定義 `pyro.sample` の際に与える API になっている
* 具体的には `baseline` 引数で指定する

#### 減衰平均ベースライン

```py
z = pyro.sample("z", dist.Bernoulli(...),
                infer=dict(baseline={'use_decaying_avg_baseline': True,
                                     'baseline_beta': 0.95}))
```

* 一番シンプルなのは、$\overline{f(z;\phi)}$ の移動指数平均を取る方法

#### ニューラルベースライン

* ベースラインをニューラルネットを用いて計算する手法
    * 入力次元：観測データの数
    * 出力次元：潜在変数の数
* 詳細は [SVI Part III: ELBO Gradient Estimators — Pyro Tutorials 0.3.0 documentation](http://pyro.ai/examples/svi_part_iii.html#Neural-Baselines)

In [1]:
from __future__ import print_function
import os
import torch
import torch.distributions.constraints as constraints
import pyro
import pyro.distributions as dist
# Pyro also has a reparameterized Beta distribution so we import
# the non-reparameterized version to make our point
from pyro.distributions.testing.fakes import NonreparameterizedBeta
import pyro.optim as optim
from pyro.infer import SVI, TraceGraph_ELBO
import sys

In [2]:
max_steps = 10000

def param_abs_error(name, target):
    return torch.sum(torch.abs(target - pyro.param(name))).item()

In [3]:
class BernoulliBetaExample(object):
    def __init__(self, max_steps):
        self.max_steps = max_steps
        # モデルの超パラメータ（興味はない）
        self.alpha0 = 10.0
        self.beta0 = 10.0
        # データ
        self.data = torch.zeros(10)
        self.data[0:6] = torch.ones(6)
        self.n_data = self.data.size(0)
        # 解析的に求まる事後分布のパラメータ
        self.alpha_n = self.data.sum() + self.alpha0
        self.beta_n = - self.data.sum() + torch.tensor(self.beta0 + self.n_data)
        # 変分パラメータの初期値
        self.alpha_q_0 = 15.0
        self.beta_q_0 = 15.0
        
    def model(self, use_decaying_avg_baseline):
        # 確率変数（潜在）の定義
        f = pyro.sample("latent_fairness", dist.Beta(self.alpha0, self.beta0))
        # プレート表現
        with pyro.plate("data_plate"):
            # 確率変数（観測）の定義
            pyro.sample("obs", dist.Bernoulli(f), obs=self.data)
     
    def guide(self, use_decaying_avg_baseline):
        # 変分パラメータの定義
        alpha_q = pyro.param("alpha_q", torch.tensor(self.alpha_q_0),
                            constraint=constraints.positive)
        beta_q = pyro.param("beta_q", torch.tensor(self.beta_q_0),
                            constraint=constraints.positive)
        # ベースラインの定義
        baseline_dict = {
            "use_decaying_avg_baseline": use_decaying_avg_baseline,
            "baseline_beta": 0.90
        }
        
        # モデルの潜在変数は、ガイドですべて定義する必要がある
        # 今回は fairness だけを定義すればよく、変分パラメータに基づくベータ分布で定義した
        pyro.sample("latent_fairness",
                    NonreparameterizedBeta(alpha_q, beta_q),
                    infer=dict(baseline=baseline_dict))
        
    def do_inference(self, use_decaying_avg_baseline, tolerance=0.1):
        # パラメータ（モデルパラメータと変分パラメータ）を一旦空にする
        pyro.clear_param_store()
        # オプティマイザのセットアップ
        optimizer = optim.Adam({"lr": .0005, "betas": (0.93, 0.999)})
        svi = SVI(self.model, self.guide, optimizer, loss=TraceGraph_ELBO())
        print("Doing inference with use_decaying_avg_baseline = %s" %
             use_decaying_avg_baseline)
        
        for k in range(self.max_steps):
            svi.step(use_decaying_avg_baseline)
            if k % 100 == 0:
                print('.', end='')
                sys.stdout.flush()
                
            # 真のパラメータとの距離を計算する
            alpha_error = param_abs_error("alpha_q", self.alpha_n)
            beta_error = param_abs_error("beta_q", self.beta_n)
            
            # 収束したら終了する
            if alpha_error < tolerance and beta_error < tolerance:
                break
                
        _alpha_q = pyro.param('alpha_q').item()
        _beta_q = pyro.param('beta_q').item()
        print("")
        print("Did %d steps of inference." % (k + 1))
        print("Final absolute errors for alpha and beta are:")
        print(f"- alpha: {_alpha_q:.4f}, {alpha_error:.4f}")
        print(f"- beta: {_beta_q:.4f}, {beta_error:.4f}")
        
bbe = BernoulliBetaExample(max_steps=max_steps)

In [4]:
bbe.do_inference(use_decaying_avg_baseline=True)

Doing inference with use_decaying_avg_baseline = True
........
Did 794 steps of inference.
Final absolute errors for alpha and beta are:
- alpha: 15.9220, 0.0780
- beta: 14.0998, 0.0998


In [5]:
bbe.do_inference(use_decaying_avg_baseline=False)

Doing inference with use_decaying_avg_baseline = False
...............................
Did 3030 steps of inference.
Final absolute errors for alpha and beta are:
- alpha: 15.9013, 0.0987
- beta: 13.9610, 0.0390
